## Train baselien 0.5758

In [1]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from tqdm import tqdm

%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from utils.dcm_utils import *
from utils.nii_utils import *
from utils.train_utils import *
from utils.xray_utils import *

from PIL import Image, ImageOps

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.io import read_image
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, CenterCrop
import torchvision.transforms.functional as TF

In [3]:
DATA_DIR = "/Volumes/SSD970/"
# DATA_DIR = "/media/longyi/SSD9701/"
TRAIN_SAGITTAL_DIR = os.path.join(DATA_DIR, "sagittal_images")
TRAIN_IMAGE_DIR = os.path.join(DATA_DIR, "train_images")

In [4]:
df = pd.read_csv(os.path.join(DATA_DIR, 'train_without_ignore.csv'))
len(df)

2012

In [5]:
df.head()

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0


In [6]:
# y = some_df[["patient_overall", 'C1', 'C2', 'C3','C4','C5','C6','C7']].values
# y

In [7]:
y = df[["patient_overall", 'C1', 'C2', 'C3','C4','C5','C6','C7']].values
y

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 1]])

In [8]:
pred = np.asarray([0.645, 0.1349, 0.2474, 0.0698, 0.1016, 0.1485, 0.2413, 0.3259]).reshape(1, -1)
pred = pred.repeat(len(y), axis=0)
pred

array([[0.645 , 0.1349, 0.2474, ..., 0.1485, 0.2413, 0.3259],
       [0.645 , 0.1349, 0.2474, ..., 0.1485, 0.2413, 0.3259],
       [0.645 , 0.1349, 0.2474, ..., 0.1485, 0.2413, 0.3259],
       ...,
       [0.645 , 0.1349, 0.2474, ..., 0.1485, 0.2413, 0.3259],
       [0.645 , 0.1349, 0.2474, ..., 0.1485, 0.2413, 0.3259],
       [0.645 , 0.1349, 0.2474, ..., 0.1485, 0.2413, 0.3259]])

In [9]:
overall_loss, c_loss = weighted_loss_with_probs(torch.tensor(pred, dtype=torch.float), torch.tensor(y))
overall_loss + c_loss

tensor(0.5758)